# RAG application using open-source models
Code adapted from https://github.com/svpino/llm

In [ ]:
#!curl -fsSL https://ollama.com/install.sh | sh

>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
!pip install langchain
!pip install langchain-core
!pip install langchain-community

!pip install pypdf
!pip install langchain[docarray]
!pip install pydantic==1.10.9
!pip install pandas
!pip install chromadb
!pip install -qU langchain-text-splitters
!pip install rank_bm25

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.7/116.7 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metada

In [ ]:
#!pip install colab-xterm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.6/115.6 kB 2.4 MB/s eta 0:00:00


In [ ]:
#%load_ext colabxterm

Run these lines in the terminal:

curl -fsSL https://ollama.com/install.sh | sh

ollama serve & ollama run llama3:8b-instruct-q4_K_M

In [ ]:
#%xterm

In [ ]:
from langchain_community.llms import Ollama
#llm = Ollama(model="llama3:8b-instruct-q4_K_M")

In [ ]:
#llm.invoke("Racontez-moi une blague")

"Voici une petite blague pour vous :\n\nPourquoi l'ordinateur a-t-il fallu aller au médecin ?\n\nParce qu'il avait un bug !\n\nComment ça ?"

In [ ]:
import os
import pandas as pd
#from dotenv import load_dotenv

#load_dotenv()

#OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
#MODEL = "gpt-3.5-turbo"
#MODEL = "mixtral:8x7b"
MODEL = "llama3:8b"
MODEL = "llama3:8b-instruct-q4_0"
MODEL="llama3:8b-instruct-q4_K_M"

REPORTING_YEAR = "2023"
COMPANY_NAME = "Kering"
COMPANY_DESCRIPTION = ""


# Load the questions file

In [ ]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

#from langchain_openai.embeddings import OpenAIEmbeddings
#from langchain_openai.chat_models import ChatOpenAI

if MODEL.startswith("gpt"):
    model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)
    embeddings = OpenAIEmbeddings()
else:
    model = Ollama(model=MODEL)
    embeddings = OllamaEmbeddings(model=MODEL, show_progress=True, num_gpu=1)

#model.invoke("Tell me a joke")

In [ ]:
#model.invoke("Raconte-moi une blague")

'Voilà :\n\nPourquoi l\'ordinateur a-t-il fallu aller voir le médecin ?\n\nParce qu\'il avait un bug !\n\n(Sorry, it\'s a bit of a "compute-r" joke)'

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser
#chain.invoke("Tell me a joke")

In [ ]:
from langchain.prompts import PromptTemplate

template = """
You are an expert in ecology and environmental issues. Your job is to examinate environmental claims of {COMPANY_NAME} based on their financial and sustainability report for the year {REPORTING_YEAR}. You will always reply in French. Answer the question based on the context below and following the given answer format. Always add to your answer actual quotes of the full sentences you used as a source. If you can't answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question", COMPANY_NAME=COMPANY_NAME, REPORTING_YEAR=REPORTING_YEAR)

'\nYou are an expert in ecology and environmental issues. Your job is to examinate environmental claims of Kering based on their financial and sustainability report for the year 2023. You will always reply in French. Answer the question based on the context below and following the given answer format. Always add to your answer actual quotes of the full sentences you used as a source. If you can\'t answer the question, reply "I don\'t know".\n\nContext: Here is some context\n\nQuestion: Here is a question\n'

# Load GRI (RSE) questions

In [ ]:
# Read the excel file as dataframe

df_GRI = pd.read_excel("/content/df_GRI_v2.xlsx")
df_GRI.info()
df_GRI.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ID                  20 non-null     object
 1   Question            20 non-null     object
 2   RSEframe            20 non-null     object
 3   FrameNet            20 non-null     object
 4   Definition          20 non-null     object
 5   CoreFE              20 non-null     object
 6   NonCoreFE           20 non-null     object
 7   MultiQuestion       20 non-null     bool  
 8   QuestionsMultiples  10 non-null     object
 9   AnswerTemplate      20 non-null     object
 10  AnswExamples        20 non-null     object
 11  Relevantinfo        8 non-null      object
 12  UnknownExample      20 non-null     object
 13  Example_1_Kering    3 non-null      object
 14  Example_2_Kering    3 non-null      object
 15  Example_3_Kering    3 non-null      object
 16  Commentaires        3 non-nu

,ID,Question,RSEframe,FrameNet,Definition,CoreFE,NonCoreFE,MultiQuestion,QuestionsMultiples,AnswerTemplate,AnswExamples,Relevantinfo,UnknownExample,Example_1_Kering,Example_2_Kering,Example_3_Kering,Commentaires,Keywords,Reponse,RetrievedContext
0,E9,E9. L'organisation a-t-elle agi pour soutenir ...,E9_Soutenir_Resilience_Action_Climatique,Supporting,A Supporter assists to strengthen the Supporte...,"['Supported', 'Supporter']","['Purpose', 'Means', 'Manner', 'Time', 'Instru...",True,"[""La compagnie a adopté des mesures pour accro...","(""resilience"": boolean)","(""resilience"": TRUE) or (""resilience"": FALSE)",❑ Nous avons adopté des mesures pour accroî...,"(""resilience"": NaN)",Conscient du lien entre la préservation du cli...,"Enfin, le Référent climat rend compte au Conse...",Tout en se concentrant sur trois priorités thé...,Le lien entre l'Organisation et les activités ...,"adaptation climatique', 'résilience climatique'","{""La compagnie a adopté des mesures pour accro...","{""La compagnie a adopté des mesures pour accro..."
1,E10,E10. Veuillez déclarer la consommation d'énerg...,E10_Consume_Renewable_Energy_Proportion,Using_resource,An Agent has access to a finite amount of some...,"['Agent', 'Resource', 'Portion', 'Purpose', 'M...","['Time', 'Place', 'Manner']",False,NaN,"(""proportionRenouvelables"": float64)","(""proportionRenouvelables"": 84.4)",Consommation d'énergies renouvelables en % de ...,"(""proportionRenouvelables"": NaN)",76 % de la consommation d’énergie totale du Gr...,L’électricité reste la principale source d’éne...,"Par ailleurs, la part d’électricité renouvelab...",L'acteur/agent tématisé dans la phrase peut êt...,"consommations', 'électricité renouvelable', 'o...","Explication:\n\nAccording to the document, Ker...","[Document(page_content=""5.2.4.2.1 Mesure des c..."
2,E1,E1. L'entreprise a-t-elle élaboré une politiqu...,E1_Set_Standards_Environment_issues,Commitment,A Speaker makes a commitment to an Addressee t...,"['Speaker', 'Message', 'Topic', 'Medium', 'Add...","['Purpose', 'Time', 'Place', 'Manner', 'Period...",True,"[""Action climatique"", ""Eau"", ""Océans"", ""Forêts...","(""politiqueEnvironmentale"": str)","(""politiqueEnvironnementale"": ""Non, et nous n'...",Required topics:;Action climatique;Eau;Océans;...,"(""politiqueEnvironnementale"": NaN)",Dans le cadre des engagements pris par le Grou...,En coordination avec la Présidente du Comité d...,Kering a structuré ses objectifs selon le cadr...,"C'est quoi une ""politique""? Est-ce que les évé...","action climatique', 'action pour le climat', '...",{'Action climatique': 'Explication = Kering a ...,"{'Action climatique': [Document(page_content=""..."
3,E2,"E2. Au cours de la période de reporting, l'ent...",E2_Engagement_Stakeholders,Collaboration,Partner_1 and Partner_2 or a group of Partners...,"['Partner_1', 'Partner_2', 'Undertaking']","['Duration', 'Time', 'Place', 'Manner', 'Means...",True,"[""Action climatique"", ""Eau"", ""Océans"", ""Forêts...","(""engagementStakeholders"": str)",('engagementStakeholders': 'Aucun engagement ...,"Required topics:, Action climatique, Eau, Océa...","(""engagementStakeholders"": NaN)",NaN,NaN,NaN,NaN,NaN,{'Action climatique': 'Explication = Kering a ...,"{'Action climatique': [Document(page_content=""..."
4,E3,E3. Quel type de mesure l'entreprise a-t-elle ...,E3_Prevention_Impact,Preventing_or_letting,A Potential_hindrance or an Agent (via their a...,"[""Agent"", ""Potential_hindrance"", ""Event""]","['Circumstances', 'Degree', 'Event_description...",True,"[""Action climatique"", ""Eau"", ""Océans"", ""Forêts...","(""preventionImpact"": str)","(""preventionImpact"": ""Formation interne / renf...","Sujets environnementaux :, Action climatique, ...","(""preventionImpact"": NaN)",NaN,NaN,NaN,NaN,NaN,{'Action climatique': 'Explication = Kering a ...,"{'Action climatique': [Document(page_content=""..."


In [ ]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate

#from langchain_openai.embeddings import OpenAIEmbeddings
#from langchain_openai.chat_models import ChatOpenAI

if MODEL.startswith("gpt"):
    model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)
    embeddings = OpenAIEmbeddings()
else:
    model = Ollama(model=MODEL)
    embeddings = OllamaEmbeddings(model=MODEL, show_progress=True, num_gpu=1)

parser = StrOutputParser()
prompt = PromptTemplate.from_template(template)


# Load the report RSE

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
#from langchain_core.text_splitter import RecursiveCharacterTextSplitter

#loader = PyPDFLoader("keringra2023.pdf")
#loader = PyPDFLoader("Kering_DEU_2023_format_PDF.pdf", extract_images=False)
loader = PyPDFLoader("/content/240326_hermes_urd2023_fr.pdf", extract_images=False)

pages = loader.load_and_split()
pages

# Extract only the DPEF report (rapport RSE) from the whole report
starting_page = 48
ending_page = 271

# Extract pages WHERE "page" key of pages' "metadata" are between starting_page and ending_page
pages_RSE = [page for page in pages if page.metadata.get("page") in range(starting_page+1, ending_page+2)]


# print last page of pages_RSE
print(pages_RSE[-1])

page_content='DOCUMENT D’ENREGISTREMENT UNIVERSEL 2023/uni00A0HERMÈS INTERNATIONAL2RESPONSABILITÉ SOCIALE, SOCIÉTALE ET ENVIRONNEMENTALE ET PERFORMANCE EXTRA ‑FINANCIÈRE\nRAPPORT D’ASSURANCE RAISONNABLE D’UN DES COMMISSAIRES AUX COMPTES DE HERMÈS INTERNATIONAL RELATIVE À CERTAINS INDICATEURS DE PERFORMANCE DURABLE\n271Indépendance et Contrôle Qualité\nNous avons réalisé notre mission dans le respect des règles d’indépendance prévues à l’article/uni00A0L./uni00A0821/uni201128 du Code de commerce, du code de \ndéontologie de la profession de Commissaire aux comptes ainsi que du «/uni00A0Code of Ethics for Professional Accountants/uni00A0» publié par l’International \nEthics Standards Board for Accountants, reposant sur les principes fondamentaux d’intégrité, d’objectivité, de compétence professionnelle et de \ndiligence, de conﬁdentialité et de conduite professionnelle.\nNous appliquons par ailleurs la norme «/uni00A0International Standard on Quality Management/uni00A01/uni00A0» qui requ

In [ ]:
# Create a text splitter from the loaded pages using RecursiveCharacterTextSplitter
#!pip install langchain-community
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

# Create text_splitter from RecursiveCharacterTextSplitter with French separators
separators = [".", "!", "?", ":", ";", "\n\n"]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200, separators=separators)
mySplittedDocs = text_splitter.split_documents(pages_RSE)

for doc in mySplittedDocs:
    # Clean line breaks doc.page_content
    doc.page_content = doc.page_content.replace("\n", " ")
    # Remove extra whitespaces
    doc.page_content = doc.page_content.replace("  ", " ")

print(mySplittedDocs[88])


vectorstore_splittedDocs = Chroma.from_documents(pages_RSE, embedding=embeddings)

retriever_splitDocs = vectorstore_splittedDocs.as_retriever()


page_content='DOCUMENT D’ENREGISTREMENT UNIVERSEL 2023/uni00A0HERMÈS INTERNATIONAL2RESPONSABILITÉ SOCIALE, SOCIÉTALE ET ENVIRONNEMENTALE ET PERFORMANCE EXTRA ‑FINANCIÈRE LES FEMMES ET LES HOMMES : LES ÉQUIPES 79RÉALISATIONS 2023 | INITIATIVES TRANSFORMANTES POUR LA COMMUNAUTÉ ET POUR L’INDIVIDU Programme d’Excellence pour les Élites de nos Quartiers Hermès a pris part pour la deuxième année au programme PEEQ qui consiste à développer des leaders, en complément à leur cursus d’études supérieures, par un parcours de développement dispensé par des consultants de haut niveau. Hermès contribue ﬁnancièrement à ce programme et accueille durant six mois deux étudiants. À terme, si une opportunité d’emploi répond à leurs aspirations, la maison peut être amenée à leur offrir un poste pérenne.Des Territoires aux Grandes écoles Favoriser l’accès des jeunes des milieux ruraux aux grandes écoles en levant les barrières liées à la fragilité ﬁnancière, à la méconnaissance des ﬁlières de l’enseignement

In [ ]:
# Test retriever
#retriever_splitDocs.invoke("zones clés pour la biodiversité")

In [ ]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever

bm25_retriever = BM25Retriever.from_documents(mySplittedDocs)
# Set the k parameter to 2 which means that the top 2 documents will be retrieved
bm25_retriever.k = 2
ensemble_retriever_SplitDocs = EnsembleRetriever(retrievers=[bm25_retriever, retriever_splitDocs], weights=[0.5, 0.5])

ensemble_retriever_SplitDocs.invoke("Superficie (en hectares) des sites possédés, loués ou gérés à proximité d'aires protégées et / ou de zones clés pour la biodiversité (KBA).")

# Run queries on LLM
It might take around 130 sec / single question on Llama3 8B and Mixtral 7x8B.

Considering we have around 60 questions in total (some single, some multiple), this could take more than 4 hours on CPU. No GPU is supported on this notebook, but it could be.

In [ ]:
# Corrected with context and question

from operator import itemgetter
import ast
#install tqdm in pip
!pip install tqdm
from tqdm import tqdm

def combine_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def model_(question, answer_template, answer_examples, unknown_examples):


    template_base_french = f""""Vous êtes un expert en écologie et environnement de l'entreprise {COMPANY_NAME}. Votre travail est de répondre à la question donnée ci-après sur la base du contexte donné ci-après datant de l'année {REPORTING_YEAR} en mentionnant la page du document source, et en respectant le "Format de réponse attendu" donné. Vous répondrez toujours en français. Si vous ne pouvez pas répondre à la question, répondez {unknown_examples}.
    """ + """
    Context: {context}

    Question: {question}
    """
    template_fr = f"{template_base_french}\nFormat de réponse attendu: Explication = (Votre réponse en trois paragraphs maximum en citant la page source) \\n\\nDonnees_extraites = {answer_template}\\n\\nAnswer_context=[citations littérales de la source que vous avez utilisées pour répondre, citation2, citation3]" + f"\n\nExemple de bonne réponse: Explication = 'L'entreprise X a exploité plus de 100 hectares de terres en Amazonie.' Donnes_extraites = {answer_examples}"


    #print(template_fr)
    prompt = PromptTemplate.from_template(template_fr)
    prompt.format(context="Here is the context", question=question, COMPANY_NAME=COMPANY_NAME, REPORTING_YEAR=REPORTING_YEAR)
    print(prompt.format(context="Here is the context", question=question, COMPANY_NAME=COMPANY_NAME, REPORTING_YEAR=REPORTING_YEAR))

    retrieved_docs = ensemble_retriever_SplitDocs.invoke(question)
    context = combine_docs(retrieved_docs)
    #chain = prompt | model | parser
    chain = (
    {
        #"context": itemgetter("question") | retriever,
        #"context": itemgetter("question") | ensemble_retriever | combine_docs,
        "context": itemgetter("context"),
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

    result = chain.invoke({"question": question, "context": context})
    print(result)
    return [result, retrieved_docs]



def add_responses_to_excel(input_df):
    # Create a copy of input_df
    df = input_df.copy()

    # Créer une nouvelle colonne pour stocker les réponses
    df['Reponse'] = ''
    df['RetrievedContext'] = ''

    # Parcourir chaque ligne du fichier Excel
    for index, row in tqdm(df.iterrows(), desc="Processing Excel rows"):
        rowID = str(row['ID'])
        # Check if it has multiple questions
        if row['MultiQuestion'] == 1:
            # Read the value on QuestionsMultiples on the first row as a list and store it in a variable
            multiquestions = row['QuestionsMultiples']
            # Convert the string to a list
            multiquestions = ast.literal_eval(multiquestions)

            # Empty dictionary to store the responses
            multiresponses = {}
            multiretrieved_docs = {}
            q_complement_index = 0

            for q_complement in tqdm(multiquestions, desc="Processing multiple questions"):
                # Counter of iterations
                q_complement_index += 1

                # Fusionner la question complémentaire avec la question principale
                question = row['Question'] + " " + q_complement
                # Obtenir la réponse en utilisant la fonction model
                r_complement = model_(question, row['AnswerTemplate'], row['AnswExamples'], row['UnknownExample'])
                # Stocker question: réponse dans le dictionnaire
                multiresponses[q_complement] = r_complement[0]
                # Stocker question: retrieved_docs dans le dictionnaire
                multiretrieved_docs[q_complement] = r_complement[1]

                # Export r_complement to a txt file
                with open(f"reponse_{index}_{rowID}_{q_complement_index}.txt", "w", encoding='utf-8') as file:
                    file.write(str(r_complement[0]))

                # Export retrieved_docs to a txt file
                with open(f"retrieved_context_{index}_{rowID}_{q_complement_index}.txt", "w", encoding='utf-8') as file:
                    file.write(str(r_complement[1]))

            # Ajouter le dictionnaire multiresponses à la nouvelle colonne 'reponse'
            df.at[index, 'Reponse'] = str(multiresponses)
            df.at[index, 'RetrievedContext'] = str(multiretrieved_docs)

            # Export multiretrieved_docs to a txt file
            with open(f"retrieved_context_{index}_{rowID}.txt", "w",  encoding='utf-8') as file:
                file.write(str(multiretrieved_docs))

            # Export multiresponses to a txt file with UTF-8 encoding
            with open(f"reponses_{index}_{rowID}.txt", "w", encoding='utf-8') as file:
                file.write(str(multiresponses))

        else:
            # Récupérer la question de la colonne 'question'
            response = model_(row['Question'], row['AnswerTemplate'], row['AnswExamples'], row['UnknownExample'])

            # Ajouter la réponse à la nouvelle colonne 'reponse'
            df.at[index, 'Reponse'] = response[0]
            df.at[index, 'RetrievedContext'] = response[1]

            # Export multiretrieved_docs to a txt file
            with open(f"retrieved_context_{index}_{rowID}.txt", "w",  encoding='utf-8') as file:
                file.write(str(response[1]))

            # Export multiresponses to a txt file with UTF-8 encoding
            with open(f"reponses_{index}_{rowID}.txt", "w", encoding='utf-8') as file:
                file.write(str(response[0]))

    # Enregistrer le DataFrame modifié dans un nouveau fichier Excel
    return df


OllamaEmbeddings:   0%|          | 0/367 [06:24<?, ?it/s]


In [ ]:
# Create blank df for the results
df_output = add_responses_to_excel(df_GRI)
df_output.info()

# Export df_output to an excel file
df_output.to_excel("df_comms_output.xlsx", index=False)

Processing Excel rows: 0it [00:00, ?it/s]
Processing multiple questions:   0%|          | 0/5 [00:00<?, ?it/s]

"Vous êtes un expert en écologie et environnement de l'entreprise Kering. Votre travail est de répondre à la question donnée ci-après sur la base du contexte donné ci-après datant de l'année 2023 en mentionnant la page du document source, et en respectant le "Format de réponse attendu" donné. Vous répondrez toujours en français. Si vous ne pouvez pas répondre à la question, répondez ("resilience": NaN).
    
    Context: Here is the context

    Question: E9. L'organisation a-t-elle agi pour soutenir l'adaptation et la résilience au action climatique ? La compagnie a adopté des mesures pour accroître la résilience de l'ensemble de l'organisation face au changement climatique?
    
Format de réponse attendu: Explication = (Votre réponse en trois paragraphs maximum en citant la page source) \n\nDonnees_extraites = ("resilience": boolean)\n\nAnswer_context=[citations littérales de la source que vous avez utilisées pour répondre, citation2, citation3]

Exemple de bonne réponse: Explication

# Zero-shot Frame elements extraction

In [ ]:
# Lisez le fichier Excel avec les réponses
df_frames = pd.read_excel("df_frames_sent.xlsx")

# Afficher le df.info
df_frames.info()

# Output folder to be used as input of answers from the RAG model
output_folder = "output"

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ID                  20 non-null     object
 1   Question            20 non-null     object
 2   RSEframe            20 non-null     object
 3   FrameNet            20 non-null     object
 4   Definition          20 non-null     object
 5   CoreFE              20 non-null     object
 6   NonCoreFE           20 non-null     object
 7   MultiQuestion       20 non-null     bool  
 8   QuestionsMultiples  10 non-null     object
 9   AnswerTemplate      20 non-null     object
 10  AnswExamples        20 non-null     object
 11  Relevantinfo        8 non-null      object
 12  UnknownExample      20 non-null     object
 13  Example_1_Kering    3 non-null      object
 14  Example_2_Kering    3 non-null      object
 15  Example_3_Kering    3 non-null      object
 16  Commentaires        3 non-nu

In [ ]:
# Create a new Ollama chain to extract the frame elements and trigger words from each sentence in df_frames
def model_frames(sent, frame_description, frame_name, frame_elements):


    template_base_french = """"
    Vous êtes un expert en linguistique, rôles sémantiques et sémantique de cadres (cadre sémantique = semantic frame, a kind of event). Un élément de cadre (Frame element) est un rôle sémantique, par example: Agent, Récipient, Instrument, Lieu, Cause. Votre travail comprend 3 tâches à faire sur la phrase "{sentence}": 1, extraire les """ + f"""éléments de l'action "{frame_name}" (description = "{frame_description}") qui sont présents dans la phrase seulement parmi les catégories d'éléments suivantes "{frame_elements}"; 2, indiquez la position de début (start_index = starting index in the Sentence string) et de fin (end_index = ending index in the Sentence string) de chaque élément comme si c'était une entité nommée spaCy (['Lieu', 21, 33]) ; 3, extrayez le mot déclencheur (a noun or non-auxiliary verb in the sentence but never a preposition nor a grammar word, it can be simple or compound and it triggers the action related to the Frame_name "{frame_name}") en fonction de la description du cadre sémantique. Vous répondrez toujours en français. Si aucun élément de cadre sémantique n'est présent dans la phrase, répondez "Je ne sais pas".
    """
    template_fr = f"{template_base_french}\nFormat de réponse attendu: Task1_FrameElements = ['element_category':'cited element from sentence', 'element2_category':'second element']. Task2_Indexes = [[21,25], [35, 74]]. Task3_TriggerWord = 'cited trigger word'"

    prompt = PromptTemplate.from_template(template_fr)
    prompt.format(frame_description=frame_description, sentence=sent)
    print(prompt.format(frame_description=frame_description, sentence=sent))

    chain = (
    {
        "frame_description": itemgetter("frame_description"),
        "sentence": itemgetter("sentence"),
    }
    | prompt
    | model
    | parser
)

    result = chain.invoke({"sentence": sent, "frame_description": frame_description})
    print(result)
    return result


In [ ]:
# Function to apply the model_frames function to each row in the dataframe df_frames
def extract_frame_elements(df):
    # Create a copy of the input dataframe
    df_results = df.copy()

    # Open the file in append mode
    with open('output/resultFrames.txt', 'a') as f:
        # Iterate over each row in the dataframe
        for index, row in df.iterrows():
            # Call the model_frames function with the values in the row
            result = model_frames(row['Sentence'], row['FrameDefinition'], row['Frame'], row['FrameElements'])

            # Write the index, Frame, and result to the file, separated by tabs
            f.write(f"{str(index)}\t{row['Frame']}\t{str(result)}\n")

            # Store the result in the corresponding Reponse column
            df_results.at[index, 'Reponse'] = str(result)

    return df_results

# Apply the extract_frame_elements function to the test_df
df_results = extract_frame_elements(df_frames)
df_results
# Export df_results to exce with headers
df_results.to_excel("df_frame_results.xlsx", index=False)